In [1]:
import pickle
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta



In [2]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
trip_table_fixed =  pd.read_csv(r'trip_table_fixed.csv', sep=",", engine='python')

In [3]:
########OPTIONAL##########
#select only the 2 hour time frame of 7-9 am from trip_table_fixed
trip_table_fixed = trip_table_fixed[trip_table_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]

In [4]:
#create graph
graph = {}
unique_table=trip_table_fixed.groupby(['full_bus_name','trip_id', 'stop_id_time']).size().reset_index()
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    graph[bus_name] = {}
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'full_bus_name']
    bus_trip_id = unique_table.loc[index, 'trip_id']
    stop_id = unique_table.loc[index, 'stop_id_time']    
    graph[bus_name][bus_trip_id] = set([])
graph_ready = copy.deepcopy(graph)
    


In [5]:
#create graph form 
#bus#:set([bus_stops])
for index, row in trip_table_fixed.iterrows():
    route_name = row['full_bus_name']
    stop_id = row['stop_id_time']
    trip_id = row['trip_id']
    try:
        graph_ready[route_name][trip_id].add(stop_id)
    except:
        print(route_name, stop_id, trip_id)
        break

In [6]:
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
for key in graph_ready:
    for trip_id in graph_ready[key]:                   
        graph_temp = sorted(list(graph_ready[key][trip_id]), key=lambda x: int(x.split('_')[1]))
        graph_ready[key][trip_id] = graph_temp


In [8]:
#graph_ready

In [9]:
count = 0 
count_list = []
graph_final = {}
node = trip_table_fixed['stop_id_time'].unique()
for stop_id_time in node:
    stop_id = stop_id_time.split('_')[0]
    stop_time = int(stop_id_time.split('_')[1]) #current time
    #wait time is 15 min 
    stop_time_30 = stop_time+1800
    graph_final[stop_id_time] = [] 
    for bus_name in graph_ready:
        for trip_id in graph_ready[bus_name]:
            stops_list = graph_ready[bus_name][trip_id]
            stops_only_list =[stops_with_time.split('_', 1)[0] for stops_with_time in stops_list]

            #gets the first hit. But a trip_id should not have multiple same stops
            try:
                index_of_stop = stops_only_list.index(stop_id)
            except:
                continue
            matching = [s for s in stops_only_list if stop_id == s]

            #check if the trip_id could possibly have two hits on a stop_id
            if len(matching) > 1: 
                print('error - trip_id has multiple same stop_ids')
                print(stop_id,bus_name,trip_id,stops_list)
                pass

            current_stop = stops_list[index_of_stop]
            current_stop_time = int(current_stop.split('_')[1])
            #check if the stop time for that bus is within 15 min wait
            if stop_time <= current_stop_time and current_stop_time <= stop_time_30: 
                #if this trip_id is the last stop, continue to the next trip_id
                try:
                    next_stop = stops_list[index_of_stop+1]
                    next_stop_id = next_stop.split('_')[0]
                    next_stop_time = next_stop.split('_')[1]
                except: 
                    continue
                travel_time = int(next_stop_time) - int(stop_time)    
                dic = {'destination': next_stop, 'travel_time': travel_time, 'trip_id': trip_id, 'bus_name': bus_name} 
                ###final step
                graph_final[stop_id_time].append(dic)
            else:
                continue #for trip_id in graph[bus_name]:

    

In [18]:
#graph_final only partial view for github upload page
#https://stackoverflow.com/questions/28704526/in-python-how-to-obtain-a-partial-view-of-a-dict
[print(v) for i, v in enumerate(graph_final.items()) if i < 10]

('1561_25223', [{'destination': '1341_25256', 'travel_time': 33, 'trip_id': 899624, 'bus_name': '22_NORTH TRANSFER: VIA MENDOTA'}, {'destination': '1341_26216', 'travel_time': 993, 'trip_id': 899625, 'bus_name': '22_NORTH TRANSFER: VIA MENDOTA'}, {'destination': '1877_26339', 'travel_time': 1116, 'trip_id': 904832, 'bus_name': '2_NORTH TRANSFER'}, {'destination': '1877_25259', 'travel_time': 36, 'trip_id': 904846, 'bus_name': '2_NORTH TRANSFER'}])
('1877_25259', [{'destination': '1759_25736', 'travel_time': 477, 'trip_id': 899623, 'bus_name': '22_NORTH TRANSFER'}, {'destination': '1759_26576', 'travel_time': 1317, 'trip_id': 899624, 'bus_name': '22_NORTH TRANSFER'}, {'destination': '1759_26360', 'travel_time': 1101, 'trip_id': 904832, 'bus_name': '2_NORTH TRANSFER'}, {'destination': '1759_25281', 'travel_time': 22, 'trip_id': 904846, 'bus_name': '2_NORTH TRANSFER'}])
('1759_25281', [])
('5100_25320', [{'destination': '1895_27042', 'travel_time': 1722, 'trip_id': 898063, 'bus_name': '17

[None, None, None, None, None, None, None, None, None, None]

In [13]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [14]:
#save_obj(graph_final, 'DFS_BUS_GRAPH')

In [16]:
#load_obj('DFS_BUS_GRAPH')